In [1]:
# Colab-ready installs. Skip if you already have these packages.
# This may take a few minutes.
!pip install -q "transformers==4.56.2" tokenizers trl==0.22.2 datasets unsloth_zoo unsloth bitsandbytes accelerate scikit-learn pyarrow==19.0.0
# If you see GPU/CUDA related errors, try restarting the runtime after install.

In [2]:
import os
import math
from pathlib import Path
from tqdm import tqdm

import torch
from datasets import load_dataset

from sklearn.metrics import accuracy_score, f1_score, classification_report

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


/home/ramnarayan.ramniwas/anaconda3/envs/semeval-oss/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.9.0+cu128
cuda available: True
Using device: cuda


In [3]:
# Login to HF if private dataset: run `huggingface-cli login` in a cell or terminal beforehand.
ds = load_dataset("DaniilOr/SemEval-2026-Task13", "A")   # per your note

# ds could be a DatasetDict or a Dataset. Normalize to train/validation splits.
if isinstance(ds, dict) or hasattr(ds, "keys"):
    keys = list(ds.keys())
    print("Dataset keys:", keys)
    if "train" in ds:
        train_ds = ds["train"]
    else:
        # if single split present, use it and create a train/validation split
        if len(keys) == 1:
            train_ds = ds[keys[0]].train_test_split(test_size=0.1, seed=3407)["train"]
            val_ds = ds[keys[0]].train_test_split(test_size=0.1, seed=3407)["test"]
        else:
            train_ds = ds[keys[0]]

    if "validation" in ds:
        val_ds = ds["validation"]
    else:
        # fallback: if user provided single dataset and we didn't already split
        if "val_ds" not in globals():
            val_ds = train_ds.train_test_split(test_size=0.1, seed=3407)["test"]

    if "test" in ds:
        test_ds = ds["test"]
else:
    # single dataset object
    full = ds
    split = full.train_test_split(test_size=0.1, seed=3407)
    train_ds = split["train"]
    val_ds = split["test"]

print("train size:", len(train_ds), "val size:", len(val_ds))
print("Columns:", train_ds.column_names)
# Quick peek
print(train_ds[0])

Dataset keys: ['train', 'validation', 'test']
train size: 500000 val size: 100000
Columns: ['code', 'generator', 'label', 'language']
{'code': "(a, b, c, d) = [int(x) for x in input().split()]\nk = input()\n(p, q, r, s) = (0, 0, 0, 0)\nfor i in k:\n\tif i == '1':\n\t\tp += 1\n\telif i == '2':\n\t\tq += 1\n\telif i == '3':\n\t\tr += 1\n\telif i == '4':\n\t\ts += 1\nprint(a * p + b * q + c * r + d * s)\n", 'generator': 'human', 'label': 0, 'language': 'Python'}


In [4]:
from unsloth import FastLanguageModel

# Choose model — 4-bit prequantized recommended on Colab for memory reasons:
MODEL_NAME = "unsloth/gpt-oss-20b"  # change if you prefer another
MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = True  # set False if you want full precision (requires much more VRAM)

print("Loading model/tokenizer (this downloads weights)...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    dtype = None,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = LOAD_IN_4BIT,
    full_finetuning = False,  # we will use PEFT/LoRA below
    offload_embedding = True,
)
print("Model + tokenizer loaded.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Loading model/tokenizer (this downloads weights)...
==((====))==  Unsloth 2025.11.4: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    NVIDIA RTX 5000 Ada Generation. Num GPUs = 1. Max memory: 31.6 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
def build_texts(batch):
    codes = batch.get("code", [])
    labels = batch.get("label", [])
    langs = batch.get("language", [])
    out_texts = []
    for i, code in enumerate(codes):
        label = labels[i] if i < len(labels) else 0
        lang = langs[i] if i < len(langs) else "Unknown"
        prompt = f"""
You are a code origin classifier. Your task is to determine whether the following code was written by a human or generated by a machine.

Output rule:
- Respond with EXACTLY ONE WORD: "human" or "machine".
- Do not include any explanations, punctuation, or extra text.

Guidelines for reasoning:
1. Naming: Human code shows mixed styles or domain-specific names. Machine code uses consistent, verbose, or generic names.
2. Comments: Human comments are sparse or informal. Machine comments are detailed, redundant, or uniformly formatted.
3. Structure: Human code may have shortcuts or irregularities. Machine code is overly structured or templated.
4. Formatting: Human code has minor inconsistencies. Machine code follows perfect style rules.
5. Logic: Human logic is pragmatic and iterative. Machine logic is overly complete or formal.

Language: {lang}

Code:
```{lang.lower()}
{code}
Output your decision (human or machine):
"""

        assistant_reply = "machine" if int(label) == 1 else "human"
        convo = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": assistant_reply},
        ]
        text = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        out_texts.append(text)
    return {"text": out_texts}

# Apply mapping (batched). This will add a `text` column that SFTTrainer expects.
train_ds = train_ds.map(build_texts, batched=True, batch_size=128)
val_ds = val_ds.map(build_texts, batched=True, batch_size=128)

print("Formatting done. Example formatted text (train):")
print(train_ds[0]["text"][:1000].replace("\n", "\\n")[:1000])

In [ ]:
print("\n" + "="*80)
print("CHECKING CHAT TEMPLATE FORMAT")
print("="*80)
example_text = train_ds[0]["text"]
print("First 1500 characters of formatted example:")
print(example_text[:6000])
print("\n" + "="*80)
print("IMPORTANT: Look at the output above and identify the EXACT markers")
print("for where the assistant response starts. Update gpt_oss_kwargs below.")
print("="*80 + "\n")

In [ ]:
# Attach PEFT/LoRA adapters to the loaded model (only a small fraction of params will be trained)
model = FastLanguageModel.get_peft_model(
    model,
    r = 12,  # try 8-32 depending on capacity
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("PEFT/LoRA adapters attached.")


In [ ]:
from trl import SFTConfig, SFTTrainer
from unsloth.chat_templates import train_on_responses_only

sft_args = SFTConfig(
    per_device_train_batch_size = 1,   # lower for Colab; increase for bigger GPUs
    gradient_accumulation_steps = 4,
    warmup_steps = 4,
    max_steps = 100,
    # num_train_epochs = 1,              # small demo run; set higher or use num_train_epochs
    learning_rate = 2e-4,
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
    report_to = "none",
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    args = sft_args,
)

# Use the correct assistant marker that appears in your formatted text
gpt_oss_kwargs = dict(
    instruction_part = "<|start|>user<|message|>",
    response_part    = "<|start|>assistant<|message|>",   # <-- corrected
)

trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)


print("\n" + "="*80)
print("MASKING VERIFICATION")
print("="*80)

def verify_masking(trainer, indices=[0, 10, min(100, len(trainer.train_dataset)-1)]):
    """Verify that only assistant responses are being trained on."""
    for idx in indices:
        if idx >= len(trainer.train_dataset):
            continue

        print(f"\n--- Example {idx} ---")

        # Show full input
        input_ids = trainer.train_dataset[idx]["input_ids"]
        full_text = tokenizer.decode(input_ids)
        print("Full conversation (last 300 chars):")
        print(full_text[-300:])

        # Show what's being trained on (should be ONLY "human" or "machine")
        labels = trainer.train_dataset[idx]["labels"]
        trained_tokens = [
            tokenizer.pad_token_id if x == -100 else x
            for x in labels
        ]
        trained_text = tokenizer.decode(trained_tokens).replace(
            tokenizer.pad_token, " "
        ).strip()

        print("\nWhat model is trained on (should be ONLY 'human' or 'machine'):")
        print(f"'{trained_text}'")

        non_masked_count = sum(1 for x in labels if x != -100)
        total_count = len(labels)
        print(f"Non-masked tokens: {non_masked_count}/{total_count}")

        # CRITICAL CHECK: Should be 1-5 tokens max (just "human" or "machine")
        if non_masked_count > 10:
            print("⚠️  WARNING: Too many non-masked tokens! Masking may not be working.")
            print("   Expected: 1-5 tokens (just the answer)")
            print(f"   Got: {non_masked_count} tokens")
        else:
            print("✓ Masking looks correct!")

verify_masking(trainer)

In [ ]:
def count_examples_with_labels(ds, sample_n=2000):
    # checks first sample_n examples (or entire ds if smaller)
    N = min(len(ds), sample_n)
    has = 0
    for i in range(N):
        lbls = ds[i]['labels']
        if any([x != -100 for x in lbls]):
            has += 1
    return has, N

has, total = count_examples_with_labels(trainer.train_dataset, sample_n=2000)
print(f"{has}/{total} examples have at least one non- -100 label.")

In [ ]:
print("Starting training...")
trainer_stats = trainer.train()
print("Training finished.")
print(trainer_stats)


In [ ]:
SAVE_DIR = "finetuned_gptoss_lora_2"
print("Saving adapters to", SAVE_DIR)
model.save_pretrained(SAVE_DIR)
print("Saved.")
# Optionally push to hub: model.push_to_hub("hf_username/repo", token="hf_...")


In [ ]:
!zip -r /content/finetuned_gptoss_lora_2.zip /content/finetuned_gptoss_lora_2

In [ ]:
from google.colab import files

files.download('finetuned_gptoss_lora_2.zip')

### Evaluation:

In [ ]:
!unzip -q /content/outputs.zip -d /content/outputs

In [ ]:
!unzip -q /content/finetuned_gptoss_lora_2.zip -d /

In [ ]:
!zip -r /content/outputs.zip /content/outputs

In [ ]:
!pip install -q "transformers==4.56.2" tokenizers datasets unsloth_zoo unsloth bitsandbytes accelerate scikit-learn pyarrow==19.0.0

import torch
from datasets import load_dataset
from unsloth import FastLanguageModel
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import numpy as np
from tqdm import tqdm

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
BASE_MODEL = "unsloth/gpt-oss-20b"  # same as training
ADAPTER_PATH = "finetuned_gptoss_lora_2"  # path to your saved adapters
MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = True

print("Loading base model + adapters...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = ADAPTER_PATH,  # this loads base model + adapters
    dtype = None,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = LOAD_IN_4BIT,
)

# Enable inference mode for faster generation
FastLanguageModel.for_inference(model)
print("Model loaded and ready for inference.")

In [ ]:
ds = load_dataset("DaniilOr/SemEval-2026-Task13", "A")

# Get validation split (same logic as training)
if isinstance(ds, dict) or hasattr(ds, "keys"):
    keys = list(ds.keys())
    print("Dataset keys:", keys)
    # if "validation" in ds:
    #     val_ds = ds["validation"]
    if "test" in ds:
        test_ds = ds["test"]
else:
    val_ds = ds.train_test_split(test_size=0.1, seed=3407)["test"]

print(f"Test set size: {len(test_ds)}")

In [ ]:
test_ds

In [ ]:
MAX_CODE_CHARS = 5000

def _crop_head_tail(text: str, max_chars: int) -> str:
    if len(text) <= max_chars:
        return text
    half = max_chars // 2
    return text[:half] + "\n... [truncated] ...\n" + text[-half:]

def _build_prompt(lang: str, code: str) -> str:
    # Use the SAME prompt structure as training
    prompt = f"""
            You are a code origin classifier. Your task is to determine whether the following code was written by a human or generated by a machine.

            Output rule:
            - Respond with EXACTLY ONE WORD: "human" or "machine".
            - Do not include any explanations, punctuation, or extra text.

            Guidelines for reasoning:
            1. Naming: Human code shows mixed styles or domain-specific names. Machine code uses consistent, verbose, or generic names.
            2. Comments: Human comments are sparse or informal. Machine comments are detailed, redundant, or uniformly formatted.
            3. Structure: Human code may have shortcuts or irregularities. Machine code is overly structured or templated.
            4. Formatting: Human code has minor inconsistencies. Machine code follows perfect style rules.
            5. Logic: Human logic is pragmatic and iterative. Machine logic is overly complete or formal.

            Language: {lang}

            Code:
            ```{lang.lower()}
            {code}
            Output your decision (human or machine):
            """
    return prompt

def predict_single(code, language="Unknown"):
    """
    Predict whether code is human or machine-generated.
    Returns: "human" or "machine"
    """
    # Crop code if too long
    code_snippet = _crop_head_tail(code, MAX_CODE_CHARS)

    # Build prompt
    prompt = _build_prompt(language, code_snippet)

    # Format as messages
    messages = [{"role": "user", "content": prompt}]

    # Apply chat template WITHOUT tokenizing (returns string)
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,  # Returns string, not tensors
        add_generation_prompt=True
    )

    # Tokenize with explicit truncation (CRITICAL for memory)
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,  # Truncate if too long
        max_length=MAX_SEQ_LENGTH  # Hard limit
    )

    # Move to GPU
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,  # Only need 1 word
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    # Decode only the generated tokens
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    response_text = generated_text.strip().lower()

    # Parse response
    if "machine" in response_text:
        return "machine"
    elif "human" in response_text:
        return "human"
    else:
        first_word = response_text.split()[0] if response_text.split() else ""
        if first_word in ["human", "machine"]:
            return first_word
        print(f"Warning: Unexpected output: '{response_text[:100]}'")
        return "human"

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

predictions = []
true_labels = []
raw_outputs = []

# Now run on full test set (no verbose to avoid clutter)
for idx in tqdm(range(len(test_ds)), desc="Predicting"):
    sample = test_ds[idx]
    code = sample.get("code", "")
    label = sample.get("label", 0)
    lang = sample.get("language", "Unknown")

    # Get prediction
    pred_text = predict_single(code, lang)
    raw_outputs.append(pred_text)

    # Convert to binary label
    pred_label = 1 if pred_text == "machine" else 0

    predictions.append(pred_label)
    true_labels.append(int(label))

    # Periodic memory cleanup
    if idx % 50 == 0 and idx > 0:
        torch.cuda.empty_cache()
        gc.collect()

# Validation
unexpected = sum(x not in ("human", "machine") for x in raw_outputs)
print(f"\nNon-binary outputs: {unexpected} (should be 0)")

if unexpected > 0:
    print("\nUnexpected outputs:")
    for i, out in enumerate(raw_outputs):
        if out not in ("human", "machine"):
            print(f"  Index {i}: '{out}'")

In [ ]:
# Show some examples
print("\nFirst 10 predictions:")
for i in range(min(10, len(raw_outputs))):
    correct = "✓" if predictions[i] == true_labels[i] else "✗"
    print(f"  {correct} {i}: Predicted={raw_outputs[i]}, True={'machine' if true_labels[i] == 1 else 'human'}")

# Calculate accuracy
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(true_labels, predictions)
print(f"\nAccuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=["human", "machine"]))

In [ ]:
import pandas as pd

df = pd.DataFrame({
    'ID': range(len(predictions)),
    'label': predictions
})

df.to_csv('test_predictions.csv', index=False)
print(f"CSV file created with {len(df)} rows")

In [ ]:
from google.colab import files

files.download('test_predictions.csv')